# Object Classification

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras import models, layers, utils
from tensorflow.keras import losses, metrics, optimizers
from tensorflow.keras.preprocessing import image

from sklearn.metrics import classification_report, confusion_matrix

from matplotlib import pyplot as plt

import numpy as np

In [ ]:
img_size = (299, 299, 3)
dir_img_train = "dataset/subset/train/"
dir_img_validation = "dataset/subset/validation/"
dir_img_test = "dataset/subset/test/"
batch_size = 16
lr = 1e-4
epoch = 22

## Prepare Dataset

In [ ]:
train_datagen = image.ImageDataGenerator(rescale=1. / 255,
                                         rotation_range=40,
                                         width_shift_range=0.2,
                                         height_shift_range=0.2,
                                         shear_range=0.2,
                                         zoom_range=0.2,
                                         horizontal_flip=True,
                                         fill_mode='nearest')

# Note that the validation data should not be augmented!
test_datagen = image.ImageDataGenerator(rescale=1. / 255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    # target folder yang berisi kumpulan citra
    dir_img_train,
    # semua citra diubah ukurannya menjadi 299 x 299
    target_size=img_size[:2],
    batch_size=batch_size,
    # class_mode categorical harus disesuaikan dengan fungsi loss
    # Karena fungsi loss yang digunakan adalah categorical crossentropy
    # maka class_mode = categorical
    class_mode="categorical",
    shuffle=True,
    seed=1234)

validation_generator = test_datagen.flow_from_directory(
    dir_img_validation,
    target_size=img_size[:2],
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True,
    seed=1234)

test_generator = test_datagen.flow_from_directory(dir_img_test,
                                                  target_size=img_size[:2],
                                                  batch_size=batch_size,
                                                  class_mode="categorical",
                                                  shuffle=True,
                                                  seed=1234)

## Mendefinisikan arsitektur model

In [ ]:
def create_model():
    conv_base = Xception(weights="imagenet",
                         include_top=False,
                         input_shape=img_size)

    conv_base.trainable = True

    input_layer = keras.Input(shape=img_size)
    x = conv_base(input_layer)
    x1 = layers.GlobalAveragePooling2D()(x)
    x2 = layers.GlobalMaxPooling2D()(x)
    x = layers.Concatenate()([x1, x2])
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(512, activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    output_layer = layers.Dense(12, activation="softmax")(x)

    model = keras.Model(inputs=input_layer, outputs=output_layer)

    return conv_base, model

In [ ]:
strategy = tf.distribute.MirroredStrategy()
print("Number of devices: {}".format(strategy.num_replicas_in_sync))

In [ ]:
# Open a strategy scope.
with strategy.scope():
    # Everything that creates variables should be under the strategy scope.
    # In general this is only model construction & `compile()`.
    conv_base, model = create_model()
    model.compile(loss=losses.CategoricalCrossentropy(),
                  optimizer=optimizers.Adam(learning_rate=lr),
                  metrics=[metrics.CategoricalAccuracy()])

In [ ]:
model.summary()

In [ ]:
utils.plot_model(model, show_layer_names=True, show_shapes=True)

## Training Model

In [ ]:
history = model.fit(train_generator,
                    epochs=epoch,
                    validation_data=validation_generator,
                    verbose=1)

In [ ]:
model.metrics_names

In [ ]:
accuracy = history.history['categorical_accuracy']
val_accuracy = history.history['val_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(accuracy))

plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs, accuracy, label='Training accuracy')
plt.plot(epochs, val_accuracy, label='Validation accuracy')
plt.title('Training and validation accuracyuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, loss, label='Training loss')
plt.plot(epochs, val_loss, label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

## Evaluate Model

In [ ]:
model.evaluate(validation_generator, verbose=1)

In [ ]:
y_true = list()
for idx in range(len(validation_generator.index_array)):
    img_idx = validation_generator.index_array[idx]
    y_true.append(validation_generator.labels[img_idx])

In [ ]:
y_pred_prob = model.predict(validation_generator, verbose=1)
y_pred = list()

for idx in range(len(y_pred_prob)):
    y_pred.append(np.argmax(y_pred_prob[idx]))

In [ ]:
print(
    classification_report(y_true,
                          y_pred,
                          target_names=[
                              'bicycle', 'cabinet', 'chair', 'coffe_maker',
                              'fan', 'kettle', 'lamp', 'mug', 'sofa',
                              'stapler', 'table', 'toaster'
                          ]))

In [ ]:
model.save("end_to_end_learning.h5")